

### ESE 527



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/"

training_data=pd.read_csv("/content/drive/My Drive/training_set.csv")
testing_data=pd.read_csv("/content/drive/My Drive/testing_set.csv")
#filtered_data=pd.read_csv("/content/drive/My Drive/filtered_data.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 anime.csv	    'Colab Notebooks'	'Personal Property Tax Receipt.pdf'   training_set.csv
 animelist.csv	     Dataset_ready.csv	 Resume.pdf			     'WashU  File '
"Brahma's Library"  'Janssen file '	 testing_set.csv		      xjj


In [ ]:
training_data

,user_id,anime_id,rating
0,0,3388,8
1,0,3273,3
2,0,1427,9
3,0,9725,8
4,0,4888,10
...,...,...,...
16035380,24169,6943,7
16035381,24169,8396,8
16035382,24169,6333,8
16035383,24169,1389,7


In [ ]:
testing_data

,user_id,anime_id,rating
0,0,6060,9
1,0,10898,7
2,0,8284,8
3,0,10997,5
4,0,57,9
...,...,...,...
4020953,24169,6734,7
4020954,24169,6132,7
4020955,24169,4831,8
4020956,24169,8868,7


In [ ]:
testing_data


,user_id,anime_id,rating
0,0,6060,9
1,0,10898,7
2,0,8284,8
3,0,10997,5
4,0,57,9
...,...,...,...
4020953,24169,6734,7
4020954,24169,6132,7
4020955,24169,4831,8
4020956,24169,8868,7


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


In [ ]:
class AnimeData(Dataset):

  def __init__(self, df):
    super().__init__()
    self.df = df[['user_id', 'anime_id', 'rating']]
    self.x_user_movie = list(zip(df['user_id'].values, df['anime_id'].values))
    self.y_rating = self.df.rating.values

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    return self.x_user_movie[idx], self.y_rating[idx]

  def loader (self,df, batch_size:int, num_workers:int=2):

    dataset=AnimeData(df)
    dataloader = DataLoader(dataset,batch_size , shuffle=True, num_workers=num_workers)
    return dataloader , len(dataset.df['user_id'].unique()) , len(dataset.df['anime_id'].unique())






In [ ]:
class MatrixFactorization(nn.Module):
    """ Matrix factorization model simple """
    def __init__(self, num_users, num_items, emb_dim):
        super().__init__()
        self.user_emb = nn.Embedding(num_embeddings=num_users, embedding_dim=emb_dim)
        self.item_emb = nn.Embedding(num_embeddings=num_items, embedding_dim=emb_dim)
    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        element_product = (user_emb*item_emb).sum(1)
        return element_product



In [ ]:
def train_validation(MF_instance,tr_dataloader, val_dataloader ,epoch:int ,lr, device):

  print("Training and validating the Model...")

  opt = optim.AdamW(MF_instance.parameters(), lr=lr)
  loss = nn.MSELoss()
  epoch_train_losses, epoch_val_losses = [], []

  for i in range(epoch):

    train_losses, val_losses = [], []
    MF_instance.train()
    for coordinate,rating in tr_dataloader:


      c_User = coordinate[0].to(device, dtype=torch.long)
      c_Item = coordinate[1].to(device, dtype=torch.long)
      Ratings = rating.to(device, dtype=torch.float)

      opt.zero_grad()
      Pred = MF_instance(c_User, c_Item)
      tr_loss = loss(Pred, Ratings)
      train_losses.append(tr_loss.item())
      tr_loss.backward()

      opt.step()

    MF_instance.eval()
    for coordinate,rating in val_dataloader:


      c_User = coordinate[0].to(device, dtype=torch.long)
      c_Item = coordinate[1].to(device, dtype=torch.long)
      Ratings = rating.to(device, dtype=torch.float)

      Pred = MF_instance(c_User, c_Item)
      val_loss = loss(Pred, Ratings)

      val_losses.append(val_loss.item())


    print(f"traing and validating for the {i}th epoch is finished!")

    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_losses.append(epoch_train_loss)
    epoch_val_losses.append(epoch_val_loss)
    print(f'Epoch: {i}, Train Loss: {epoch_train_loss:0.5f}, Val Loss:{epoch_val_loss:0.5f}')

In [ ]:
Anime_tr=AnimeData(training_data)
Anime_te=AnimeData(testing_data)

In [ ]:
tr_loader, user_dim_1, anime_dim_1=Anime_tr.loader(df=training_data, batch_size=40000, num_workers=4)
te_loader, user_dim_2, anime_dim_2 =Anime_te.loader(df=testing_data, batch_size=10000, num_workers=4)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
My_Model= MatrixFactorization (num_users=user_dim_1, num_items=16886, emb_dim=10)
My_Model.to(device)

MatrixFactorization(
  (user_emb): Embedding(24170, 10)
  (item_emb): Embedding(16886, 10)
)

In [ ]:
train_validation(My_Model,tr_loader, te_loader ,5 ,0.05 , device=device)

Training and validating the Model...


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


traing and validating for the 0th epoch is finished!
Epoch: 0, Train Loss: 11.14815, Val Loss:1.59925
traing and validating for the 1th epoch is finished!
Epoch: 1, Train Loss: 1.48915, Val Loss:1.44570
traing and validating for the 2th epoch is finished!
Epoch: 2, Train Loss: 1.41530, Val Loss:1.41446
traing and validating for the 3th epoch is finished!
Epoch: 3, Train Loss: 1.39655, Val Loss:1.40969
traing and validating for the 4th epoch is finished!
Epoch: 4, Train Loss: 1.38795, Val Loss:1.40712
